In [1]:
import tensorflow as tf
import numpy as np
from box_embeddings.modules.intersection.tf_intersection import TFIntersection
from box_embeddings.modules.volume.tf_volume import TFVolume
from box_embeddings.parameterizations.tf_box_tensor import TFBoxFactory
import tensorflow_probability as tfp
from tensorflow.keras import Model
from box_embeddings.modules.tf_embedding import TFBoxEmbedding

AllenNLP not available. Registrable won't work.


In [2]:
x_train=[]
y_train=[]

In [3]:
f = open("classification_samples_valid2id.txt", "r")
for x in f:
    x = x.split(" ")
    if len(x)<4:
        continue
    else:
        x_train.append([x[0],x[1]])
        y_train.append(x[3])

In [5]:
print(len(x_train))
print(len(y_train))
print(x_train[0])
print(y_train[0])

317218
317218
['61136', '13652']
1



In [41]:
def TFNNLoss(y_pred,y_orig):
    y_pred = tf.stack([tfp.math.log1mexp(y_pred),y_pred],axis=-1)
    mask = y_orig>0
    y_pred_T = tf.transpose(y_pred)
    res = y_pred_T[1]
    res = tf.where(mask,res,y_pred_T[0])
    return tf.reduce_mean(-res)

def run_optimization(x_h, x_p, y):
    with tf.GradientTape() as g:
        pred = neural_net(x_h, x_p, is_training=True)
        loss = TFNNLoss(pred, y)
        
    trainable_variables = neural_net.trainable_variables
    gradients = g.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    
def accuracy(y_pred, y_true):
    y_pred = tf.stack([tfp.math.log1mexp(y_pred),y_pred],axis=-1)
    correct_prediction = tf.equal(tf.argmax(y_pred, 0), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [42]:
class NeuralNet(Model):
    
    def __init__(self, box_factory, box_intersection, box_volume):
        super(NeuralNet, self).__init__()
        
        self.box_factory = box_factory
        self.box_intersection = box_intersection
        self.box_volume = box_volume
        self.embedder = TFBoxEmbedding(84363, 100)
        
    
    def __call__(self, head, tail, is_training=False):
        premise_embeddings = self.embedder.embeddings[head]
        hypothesis_embeddings = self.embedder.embeddings[tail] 
            
        premise_box = self.box_factory(premise_embeddings)
        hypothesis_box = self.box_factory(hypothesis_embeddings)
        
        output = self.box_volume(self.box_intersection(premise_box, hypothesis_box)) - self.box_volume(premise_box)
        
        return output

In [43]:
box_factory = TFBoxFactory('mindelta_from_vector')
box_volume = TFVolume(volume_temperature=100)
box_intersection = TFIntersection(intersection_temperature=10)

display_step=100
epochs=20


optimizer = tf.keras.optimizers.Adam(0.01)
neural_net = NeuralNet(box_factory,box_intersection,box_volume)

In [45]:
for epoch in range(1,epochs+1):
    
    for i,data in enumerate(x_train):
        
        run_optimization(x_train[i][0], x_train[i][1], y_train[i])

        if i % display_step == 0:
            pred = neural_net(x_train[i][0], x_train[i][1], is_training=True)
            loss = TFNNLoss(pred, y_train[i])
            #acc = accuracy(pred, y_train[i])
            #write(str(datetime.datetime.now())+" step: %i, loss: %f, accuracy: %f" % (step, loss, acc),"train-log.txt")
            print("epoch: %i, step: %i, loss: %f" % (epoch, i, loss))

epoch: 1, step: 0, loss: 10.157183
epoch: 2, step: 0, loss: 10.156011
